# 文件读写

## 加载和保存张量(矩阵)

In [1]:
import torch as tor
from torch import nn as tor_nn
from torch.nn import functional as tor_nn_F

导入库

In [4]:
x = tor.arange(4)
tor.save(x, 'x-file')

x2 = tor.load('x-file')
x2

tensor([0, 1, 2, 3])

存储一个张量列表，然后将他们读回内存

In [6]:
y = tor.zeros(4)
tor.save([x, y], 'xy-file') # 这里就可以看到度的是一个列表
x2, y2 = tor.load('xy-file')

(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

使用python的字典对于数据进行特殊的索引标注

In [7]:
my_dict = {'x' : x, 'y' : y}
tor.save(my_dict, 'my_dict')

my_dict2 = tor.load('my_dict')
my_dict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

## 加载与保存模型参数

我们可以看到我们加载和保存模型参数，由于模型并不是一个单独的矩阵，并且他们有一定的模型结构，所以会才有别的方式去搭建模型。

In [8]:
class MLP_RW(tor_nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = tor_nn.Linear(20, 256)
        self.output = tor_nn.Linear(256, 10)
    
    def forward(self, x):
        return self.output(tor_nn_F.relu(self.hidden(x)))
    
net = MLP_RW()
X = tor.randn(size = (2, 20))
Y = net(X)

接下来就是将模型的参数存入到一个后缀为 .params 的文件中

In [9]:
tor.save(net.state_dict(), 'mlp_rw.params')
net.state_dict()

OrderedDict([('hidden.weight',
              tensor([[ 0.1862,  0.1383, -0.1819,  ...,  0.1951,  0.1637,  0.0874],
                      [-0.0618, -0.0119,  0.0510,  ..., -0.1947, -0.1041, -0.0753],
                      [ 0.1732,  0.0181, -0.1076,  ...,  0.1266, -0.0324,  0.0499],
                      ...,
                      [-0.0564,  0.2200,  0.2235,  ...,  0.1828,  0.1041,  0.1561],
                      [-0.1013,  0.1492,  0.1231,  ...,  0.2062,  0.1642,  0.1825],
                      [ 0.1044, -0.1886, -0.0864,  ...,  0.2089,  0.0791,  0.0268]])),
             ('hidden.bias',
              tensor([-0.1690, -0.0954, -0.2136, -0.0375,  0.2088,  0.0008,  0.1445, -0.0699,
                      -0.0770,  0.0155,  0.1877,  0.2126, -0.1203,  0.0236, -0.1534, -0.0337,
                       0.1316,  0.0214, -0.0153,  0.0405, -0.1814, -0.1155,  0.1642,  0.1430,
                      -0.1244, -0.1034, -0.1084,  0.0922,  0.0287, -0.2212,  0.1513,  0.1089,
                      -0.1262,

接下来是让我们的数据进行一下回归

In [10]:
clone_mod = MLP_RW()
clone_mod.load_state_dict(tor.load("mlp_rw.params"))
clone_mod.eval()

MLP_RW(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [11]:
Y_clone = clone_mod(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

我们其实可以看到，这并不算是一个完全意义上的存模型，这个东西其实是一种参数单独剔除出来，然后骨架是还在那里的，要使用时是先实例化骨架再将参数放入其中。